In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stat

In [29]:
# Read in dataset and choose relevant columns
house_df = pd.read_csv('1976-2022-house.csv')
house_df = house_df[~house_df['special']]
house_df = house_df[['year', 'state_po', 'district', 'candidate', 'party', 'candidatevotes', 'totalvotes']]

house_df = house_df[(house_df['candidate'] != 'WRITEIN') & (house_df['candidate'] != 'SCATTERING')]
house_df['party'] = house_df['party'].astype('|S')
house_df['party'] = house_df['party'].apply(lambda x : 
                                            'DEMOCRAT' if 
                                                (('DEMOCRAT' in x) or ('LIBERAL' in x)) else 
                                                ('REPUBLICAN' 
                                                    if (('REPUBLICAN' in x) or ('CONSERVATIVE' in x))
                                                    else x)
)

# Renames state_po(stal code) column label to just state
house_df['state'] = house_df['state_po']
house_df = house_df[['year', 'state', 'district', 'candidate', 'party', 'candidatevotes', 'totalvotes']]

# Find primary party of candidates in fusion tickets
grouped = house_df.groupby(['year', 'state', 'district', 'candidate'])
house_df = house_df.loc[grouped['candidatevotes'].idxmax()]

# Find total votes for fusion tickets
house_df = house_df[['year', 'state', 'district', 'candidate', 'party', 'totalvotes']]
sums = grouped['candidatevotes'].sum().reset_index()
house_df = pd.merge(house_df, sums, 'inner', on = ['year', 'state', 'district', 'candidate'])

# Find total votes cast per party
house_df = house_df.groupby(['year', 'state', 'district', 'party', 'totalvotes'])['candidatevotes'].sum().reset_index()

# Aptly rename the candidatevotes no longer referring to candidates
house_df['votes'] = house_df['candidatevotes']
house_df = house_df[['year', 'state', 'district', 'party', 'votes', 'totalvotes']]

# Filter only top two parties
grouped = house_df.groupby(['year', 'state', 'district'])
two_party_votes = grouped['votes'].apply(lambda x : x.sort_values().tail(2)).reset_index()

# Calculate two party vote percentage
two_party_votes['two_party_pct'] = two_party_votes['votes'] / two_party_votes.groupby(['year', 'state', 'district'])['votes'].transform('sum')

house_df = pd.merge(house_df, two_party_votes, 'inner', on = ['year', 'state', 'district', 'votes'])
house_df = house_df.drop(columns= ['level_3'])

house_df

TypeError: a bytes-like object is required, not 'str'

In [12]:
two_party_votes['votes'] / two_party_votes.groupby(['year', 'state', 'district'])['votes'].transform('sum')

0        0.289986
1        0.710014
2        0.374808
3        0.625192
4        0.423953
           ...   
20038    0.698404
20039    0.344314
20040    0.655686
20041    0.263296
20042    0.736704
Name: votes, Length: 20043, dtype: float64

In [26]:
house_df[(house_df['party'] != 'DEMOCRAT') & (house_df['party'] != 'REPUBLICAN') & (house_df['two_party_pct'] > 0.1) & (house_df['year'] > 2000)].groupby('party').count()

,year,state,district,votes,totalvotes,two_party_pct
party,,,,,,
AMERICANS ELECT,1,1,1,1,1,1
CONSERVATIVE,6,6,6,6,6,6
CONSTITUTION,4,4,4,4,4,4
DEMOCRATIC-FARM-LABOR,1,1,1,1,1,1
DEMOCRATIC-FARMER-LABOR,59,59,59,59,59,59
DEMOCRATIC-NONPARTISAN LEAGUE,3,3,3,3,3,3
DEMOCRATIC-NPL,1,1,1,1,1,1
GREEN,20,20,20,20,20,20
INDEPENDENT,45,45,45,45,45,45


In [ ]:
grouped = 